Hier werden die Portfolio-Methoden angewandt und mithilfe der Performance-Metriken bewertet. Die Ergebnisse werden in CSV-Dateien festgehalten. Das Akquirieren, modifizieren und beschreiben der Daten befindet sich der Übersicht halber im Notebook Datenbasis. Die Daten werden dort als CSV exportiert und hier als Dataframe wieder eingelesen.

Die Korrektheit der implementierten Methoden und Metriken wird in dem separaten Notebook Verifizierung anhand eines kleinen Teilbestandes aufgezeigt, siehe dazu auch beigefügte Excel-Dateien, in denen die Portfolio-Gewichte "per Hand" nachgerechnet und mit den Ergebnissen aus der Python-Implementierung abgeglichen werden.

ACHTUNG: LANGE LAUFZEIT!

## Importe

In [1]:
#Import der notwendigen Bibliotheken, Installation mit pip bereits erfolgt

import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

## Einlesen der Daten

In [2]:
#Bearbeitete JSE-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'df_JSE_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [3]:
df_JSE.info

<bound method DataFrame.info of             4SI.JO   ABG.JO  ACL.JO  ACT.JO       ADH.JO  AEL.JO  \
Date                                                               
2005-06-01     NaN   8240.0  4980.0   130.0   133.701508     NaN   
2005-06-02     NaN   8250.0  4949.0   130.0   131.749695     NaN   
2005-06-03     NaN   8261.0  4949.0   130.0   131.749695     NaN   
2005-06-06     NaN   8290.0  5023.0   130.0   132.725601     NaN   
2005-06-07     NaN   8345.0  4845.0   149.0   131.749695     NaN   
...            ...      ...     ...     ...          ...     ...   
2025-05-26    70.0  16845.0   109.0   160.0  3199.000000  2350.0   
2025-05-27    70.0  16848.0   105.0   160.0  3210.000000  2350.0   
2025-05-28    73.0  17300.0   101.0   174.0  3283.000000  2355.0   
2025-05-29    70.0  17579.0   106.0   166.0  3247.000000  2362.0   
2025-05-30    70.0  17332.0   116.0   160.0  3215.000000  2358.0   

                  AFE.JO  AFH.JO  AFT.JO        AGL.JO  ...  VKE.JO   VOD.JO  \
Dat

In [4]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_SP500 = pd.read_csv(
    'df_SP500_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [5]:
df_SP500.info

<bound method DataFrame.info of                      A  AAL  AAP        AAPL        ABBV        ABNB  \
Date                                                                   
2005-06-01   17.296137  NaN  NaN    1.439286         NaN         NaN   
2005-06-02   17.303289  NaN  NaN    1.430000         NaN         NaN   
2005-06-03   17.210300  NaN  NaN    1.365714         NaN         NaN   
2005-06-06   17.539343  NaN  NaN    1.354286         NaN         NaN   
2005-06-07   17.324751  NaN  NaN    1.305000         NaN         NaN   
...                ...  ...  ...         ...         ...         ...   
2025-05-23  108.529999  NaN  NaN  195.270004  183.259995  126.720001   
2025-05-27  111.260002  NaN  NaN  200.210007  185.720001  129.399994   
2025-05-28  110.879997  NaN  NaN  200.419998  183.089996  128.669998   
2025-05-29  113.279999  NaN  NaN  199.949997  185.619995  128.360001   
2025-05-30  111.919998  NaN  NaN  200.850006  186.110001  129.000000   

                   ABT       AC

In [6]:
#Reduzierter SP500-Bestand ohne ausgeschiedene Unternehmen und auf Größe des JSE runtergebrochen

df_SP500_reduziert = pd.read_csv(
    'sp500_gefiltert_neuneu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    dayfirst=True
)

In [7]:
df_SP500_reduziert.info

<bound method DataFrame.info of                      A        AAPL         ABT         ACN         ADI  \
Date                                                                     
2005-06-01   17.296137    1.439286         NaN         NaN         NaN   
2005-06-02   17.303289    1.430000         NaN         NaN         NaN   
2005-06-03   17.210300    1.365714         NaN         NaN         NaN   
2005-06-06   17.539343    1.354286         NaN         NaN         NaN   
2005-06-07   17.324751    1.305000         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2025-05-23  108.529999  195.270004  131.300003  309.579987  210.470001   
2025-05-27  111.260002  200.210007  132.940002  315.429993  216.889999   
2025-05-28  110.879997  200.419998  132.020004  315.989990  215.619995   
2025-05-29  113.279999  199.949997  132.850006  317.730011  215.750000   
2025-05-30  111.919998  200.850006  133.580002  316.820007  213.979996   

     

## Definition der Methoden und Metriken

#### Equally-Weighted Portfolio

In [8]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

#### Minimum-Variance Portfolio

In [9]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

#### Maximum Diversification Portfolio

In [10]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [11]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=matrix, method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

#### Inverse Volatility Portfolio

In [12]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

#### Equal Risk Contribution Portfolio

In [13]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [14]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

#### Hierarchical Risk Parity

In [15]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [16]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [17]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [18]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [19]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [20]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

#### Hierarchical Equal Risk Contribution Portfolio

In [21]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

In [22]:
def herc_ledoit (returns):

    #Building the portfolio object
    port = hc(returns=returns)
    
    shrinkage_herc = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix_herc = pd.DataFrame(
        shrinkage_herc.covariance_,
        index = returns.columns,
        columns = returns.columns)

    #Estimate optimal portfolio:

    model='HERC'
    #correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          custom_cov = shrinkage_matrix_herc,
                          codependence='custom_cov',
                          rm=rm,
                          rf=rf,
                          method_cov='custom_cov',   #gemäß https://riskfolio-lib.readthedocs.io/en/latest/hcportfolio.html
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

#### Vergleichs-Metriken

In [23]:
def gesamt_return(gewichte, returns):

    daily_returns = returns @ gewichte
    total = np.prod(1 + daily_returns) - 1
    
    return total

In [24]:
def portfolio_return (gewichte, returns):
    
    avg_returns = []
    avg_returns = returns.mean()
    
    pf_return = sum([gewicht * rendite for gewicht, rendite in zip(gewichte, avg_returns)])
    
    return pf_return

In [25]:
def portfolio_variance (gewichte, returns):
    daily_returns = []
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    pf_var = np.var(daily_returns)
    
    return pf_var

In [26]:
def portfolio_volatility (gewichte, returns):
    variance = portfolio_variance(gewichte, returns)
    volatility = np.sqrt(variance)
    
    return volatility

In [27]:
def annualized_return(gewichte, returns):
    daily_returns = returns @ gewichte
    annual = (np.prod(1 + daily_returns))**(252/daily_returns.shape[0]) - 1
    
    return annual

In [28]:
def annualized_portfolio_return(gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    
    return pf_return*252

In [29]:
def annualized_portfolio_volatility(gewichte, returns):
    pf_volatility = portfolio_volatility(gewichte, returns)
    
    return pf_volatility*np.sqrt(252)

In [30]:
def sharpe_ratio (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [31]:
def sharpe_ratio_one (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [32]:
def sharpe_ratio_two (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [33]:
def annualized_sharpe_ratio (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [34]:
def annualized_sharpe_ratio_one (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [35]:
def annualized_sharpe_ratio_two (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [36]:
def sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [37]:
def sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [38]:
def sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [39]:
def annualized_sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [40]:
def annualized_sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [41]:
def annualized_sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [42]:
def maximum_drawdown (gewichte, returns):
    daily_returns = returns @ gewichte
    returns_kumuliert = (1 + daily_returns).cumprod()
    hochpunkte = returns_kumuliert.cummax()
    
    drawdowns = (returns_kumuliert - hochpunkte) / hochpunkte
    max_drawdown = drawdowns.min()
    
    #print(drawdowns)
    return max_drawdown

In [43]:
def calmar_ratio (gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    max_drawdown = maximum_drawdown(gewichte, returns)
    
    calmar_ratio = pf_return / (-max_drawdown)
    
    return calmar_ratio

In [44]:
def value_at_risk_func(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = np.percentile(sorted_daily_returns, 100*0.05)
    return value_at_risk

In [45]:
#https://github.com/malctaylor15/VaR-and-Expected-Shortfall-Python/blob/master/Expected%20Shortfall%20and%20Value%20at%20Risk.ipynb

def expected_shortfall(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = value_at_risk_func(gewichte, returns)
    
    expected_shortfall = daily_returns[daily_returns <= value_at_risk].mean()
    
    return expected_shortfall

#### Sammel-Funktionen zum Berechnen und Speichern der Metriken

In [46]:
def metriken_equally_weighted(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile1 = {"Methode": "Equally-Weighted",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile1[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile1])], ignore_index=True)

In [47]:
def metriken_minimum_variance(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile2 = {"Methode": "Minimum_Variance",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile2[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile2])], ignore_index=True)

In [48]:
def metriken_max_diversification(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile3 = {"Methode": "Max-Diversification",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile3[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile3])], ignore_index=True)

In [49]:
def metriken_inverse_volatility(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile4 = {"Methode": "Inverse-Volatility",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile4[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile4])], ignore_index=True)

In [50]:
def metriken_risk_parity(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile5 = {"Methode": "Risk-Parity",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile5[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile5])], ignore_index=True)

In [51]:
def metriken_hrp(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile6 = {"Methode": "HRP",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile6[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile6])], ignore_index=True)

In [52]:
def metriken_herc(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile7 = {"Methode": "HERC",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile7[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile7])], ignore_index=True)

## Erzeugung Ergebnisse S&P500

In [53]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [54]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 2
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    gewichte_equally_weighted = equally_weighted(training_daten)
    gewichte_min_variance= min_variance_opt(cov_matrix)
    gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": gewichte_equally_weighted,
    "Minimum-Variance": gewichte_min_variance,
    "Maximum-Diversification": gewichte_max_diversification,
    "Inverse-Volatility": gewichte_inverse_volatility,
    "Equal-Risk-Contribution": gewichte_risk_parity,
    "HRP": gewichte_hrp,
    "HERC": gewichte_herc
}
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df = pd.concat(portfolio_returns_list)
portfolio_returns_df = portfolio_returns_df.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.560718656371804e-05
            Iterations: 299
            Function evaluations: 78936
            Gradient evaluations: 299
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0661056745424897
            Iterations: 87
            Function evaluations: 22007
            Gradient evaluations: 83
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6806677955014559e-09
            Iterations: 21
            Function evaluations: 5544
            Gradient evaluations: 21
2005-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.690668950260142e-05
            Iterations: 318
            Function evaluations: 84588
            Gradient evaluations: 318
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.609616389400448
            Iterations: 35
            Function evaluations: 10218
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.101271457203561e-10
            Iterations: 66
            Function evaluations: 19206
            Gradient evaluations: 66
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.515465372412421e-05
            Iterations: 234
            Function evaluations: 68094
            Gradient evaluations: 234
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.554867540794491
            Iterations: 55
            Function evaluations: 14905
            Gradient evaluations: 51
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.540393150899734e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.58014660323221e-10
            Iterations: 78
            Function evaluations: 24258
            Gradient evaluations: 78
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.697985496953368e-05
            Iterations: 262
            Function evaluations: 82530
            Gradient evaluations: 262
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3123645265484116
            Iterations: 40
            Function evaluations: 11380
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1373323212787251e-09
            Iterations: 56
            Function evaluations: 17640
            Gradient evaluations: 56
2009-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.9332

2011-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.7554615338287796e-05
            Iterations: 214
            Function evaluations: 70406
            Gradient evaluations: 214
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9702056130066234
            Iterations: 27
            Function evaluations: 8902
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.756578081164697e-10
            Iterations: 61
            Function evaluations: 20070
            Gradient evaluations: 61
2011-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.779886606901843e-05
            Iterations: 233
            Function evaluations: 76890
            Gradient evaluations: 233
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9128283698343327
            Iterations: 76
            Function evaluations: 25205
            Gradient evaluations: 72
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.4862375709749555e-09
            Iterations: 8
            Function evaluations: 2792
            Gradient evaluations: 8
2013-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.775029877717052e-05
            Iterations: 363
            Function evaluations: 127413
            Gradient evaluations: 363
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0212686964126294
            Iterations: 59
            Function evaluations: 19357
            Gradient evaluations: 55
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0511037334251625e-09
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7391516005503752e-09
            Iterations: 45
            Function evaluations: 16741
            Gradient evaluations: 45
2015-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.5366183812349686e-05
            Iterations: 298
            Function evaluations: 111453
            Gradient evaluations: 298
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8796359379731653
            Iterations: 52
            Function evaluations: 17996
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1270262210501566e-09
            Iterations: 55
            Function evaluations: 20570
            Gradient evaluations: 55
2015-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2924924725675058e-05
            Iterations: 369
            Function evaluations: 150552
            Gradient evaluations: 369
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.236792957659217
            Iterations: 62
            Function evaluations: 23719
            Gradient evaluations: 58
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6119501072000555e-09
            Iterations: 20
            Function evaluations: 8160
            Gradient evaluations: 20
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.8323271810440742e-05
            Iterations: 394
            Function evaluations: 161146
            Gradient evaluations: 394
Optimization terminated successfully    (Exit mode 0)
            Current function value: -

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.997204323420157
            Iterations: 27
            Function evaluations: 9724
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.408001330022897e-10
            Iterations: 89
            Function evaluations: 37558
            Gradient evaluations: 89
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000159027604595843
            Iterations: 207
            Function evaluations: 87976
            Gradient evaluations: 207
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.028847680699448
            Iterations: 25
            Function evaluations: 10647
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1343457402598383e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.035898366480985e-10
            Iterations: 56
            Function evaluations: 24976
            Gradient evaluations: 56
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.336828812328271e-05
            Iterations: 238
            Function evaluations: 106386
            Gradient evaluations: 238
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3472929859773752
            Iterations: 40
            Function evaluations: 16124
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0865034325044437e-09
            Iterations: 55
            Function evaluations: 24585
            Gradient evaluations: 55
2022-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.50

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4294372767715442e-05
            Iterations: 321
            Function evaluations: 149908
            Gradient evaluations: 321
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4.357105474789429
            Iterations: 94
            Function evaluations: 42146
            Gradient evaluations: 90
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.940671184612122e-09
            Iterations: 33
            Function evaluations: 15411
            Gradient evaluations: 33
2024-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4729774907206057e-05
            Iterations: 385
            Function evaluations: 179410
            Gradient evaluations: 385
Positive directional derivative for linesearch    (Exit mode 8)
            Curre

In [55]:
#Exportieren nach CSV
portfolio_returns_df.to_csv("renditensp500_2Monate_f.csv")

## Ergebnis-Erzeugung JSE

In [56]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [57]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 2
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": JSE_gewichte_equally_weighted,
    "Minimum-Variance": JSE_gewichte_min_variance,
    "Maximum-Diversification": JSE_gewichte_max_diversification,
    "Inverse-Volatility": JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": JSE_gewichte_risk_parity,
    "HRP": JSE_gewichte_hrp,
    "HERC": JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_JSE = portfolio_returns_df_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3134440810290774e-05
            Iterations: 152
            Function evaluations: 10184
            Gradient evaluations: 152
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.653137425667317
            Iterations: 53
            Function evaluations: 3590
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1202687138208137e-09
            Iterations: 44
            Function evaluations: 2948
            Gradient evaluations: 44
2005-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.2112396294223096e-05
            Iterations: 132
            Function evaluations: 8976
            Gradient evaluations: 132
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.24

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.877491757511488
            Iterations: 44
            Function evaluations: 3432
            Gradient evaluations: 44
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.683014382006474e-09
            Iterations: 43
            Function evaluations: 3311
            Gradient evaluations: 43
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.493057894033677e-05
            Iterations: 133
            Function evaluations: 10640
            Gradient evaluations: 133
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0375286887945956
            Iterations: 43
            Function evaluations: 3480
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.2125919608147287e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.837832603079748e-09
            Iterations: 76
            Function evaluations: 6536
            Gradient evaluations: 76
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6793775693611726e-05
            Iterations: 208
            Function evaluations: 17473
            Gradient evaluations: 208
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.544390252349358
            Iterations: 57
            Function evaluations: 4503
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.316071954319211e-09
            Iterations: 56
            Function evaluations: 4705
            Gradient evaluations: 56
2009-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2508919

2011-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.224677230603339e-05
            Iterations: 215
            Function evaluations: 19781
            Gradient evaluations: 215
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.495749693532197
            Iterations: 55
            Function evaluations: 5115
            Gradient evaluations: 55
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.5527298151406096e-09
            Iterations: 57
            Function evaluations: 5245
            Gradient evaluations: 57
2011-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.850105490812525e-06
            Iterations: 248
            Function evaluations: 22568
            Gradient evaluations: 248
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.168

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.558173688946966
            Iterations: 76
            Function evaluations: 7917
            Gradient evaluations: 76
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6775506899368875e-09
            Iterations: 33
            Function evaluations: 3399
            Gradient evaluations: 33
2013-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.650210676164808e-06
            Iterations: 219
            Function evaluations: 22776
            Gradient evaluations: 219
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.68674561942134
            Iterations: 67
            Function evaluations: 7048
            Gradient evaluations: 67
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.895644754795919e-09
           

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7873585444051588e-08
            Iterations: 137
            Function evaluations: 16029
            Gradient evaluations: 137
2015-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2087583309331887e-05
            Iterations: 198
            Function evaluations: 23167
            Gradient evaluations: 198
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.7806908765104135
            Iterations: 100
            Function evaluations: 11380
            Gradient evaluations: 96
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3896128488184162e-08
            Iterations: 114
            Function evaluations: 13338
            Gradient evaluations: 114
2015-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2246580640706588e-05
            Iterations: 258
            Function evaluations: 33798
            Gradient evaluations: 258
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.8666158137326905
            Iterations: 82
            Function evaluations: 10733
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.457373620471908e-09
            Iterations: 52
            Function evaluations: 6812
            Gradient evaluations: 52
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6313420519127e-05
            Iterations: 258
            Function evaluations: 34830
            Gradient evaluations: 258
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.854

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7952538812954333
            Iterations: 62
            Function evaluations: 8060
            Gradient evaluations: 58
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.129981686849365e-10
            Iterations: 93
            Function evaluations: 12834
            Gradient evaluations: 93
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.099400897374872e-05
            Iterations: 174
            Function evaluations: 24708
            Gradient evaluations: 174
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.849662376664347
            Iterations: 67
            Function evaluations: 9005
            Gradient evaluations: 63
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2225191339160474e-08
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6778631791198616e-09
            Iterations: 55
            Function evaluations: 8306
            Gradient evaluations: 55
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5527687349963467e-05
            Iterations: 220
            Function evaluations: 33440
            Gradient evaluations: 220
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.184237198969174
            Iterations: 107
            Function evaluations: 15841
            Gradient evaluations: 103
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.3596794764361764e-09
            Iterations: 44
            Function evaluations: 6689
            Gradient evaluations: 44
2022-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.37

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.090650657560652e-05
            Iterations: 274
            Function evaluations: 41648
            Gradient evaluations: 274
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -6.42072094048574
            Iterations: 124
            Function evaluations: 18525
            Gradient evaluations: 120
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.288669387680618e-09
            Iterations: 49
            Function evaluations: 7448
            Gradient evaluations: 49
2024-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0519396753445875e-05
            Iterations: 257
            Function evaluations: 39064
            Gradient evaluations: 257
Positive directional derivative for linesearch    (Exit mode 8)
            Current 

In [58]:
#Exportieren nach CSV
portfolio_returns_df_JSE.to_csv("returnsJSE_2Monate_f.csv")

## Ledoit-Wolf Shrinkage

In [59]:
#Definition von Ledoit-Shrinkage

def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    return shrinkage_matrix

### Auswertung S&P500 mit Ledoit Wolf

In [60]:
returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [61]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 2
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_gewichte_min_variance,
    "Maximum-Diversification": shrinked_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_gewichte_risk_parity,
    "HRP": shrinked_gewichte_hrp,
    "HERC": shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked = portfolio_returns_df_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5288862200654737e-05
            Iterations: 308
            Function evaluations: 81312
            Gradient evaluations: 308
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1767340633144645
            Iterations: 80
            Function evaluations: 20138
            Gradient evaluations: 76
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.367469318835228e-09
            Iterations: 9
            Function evaluations: 2376
            Gradient evaluations: 9
2005-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.67889435502806e-05
            Iterations: 300
            Function evaluations: 79800
            Gradient evaluations: 300
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9599

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6422651399028068
            Iterations: 42
            Function evaluations: 12262
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1484385309851184e-09
            Iterations: 57
            Function evaluations: 16588
            Gradient evaluations: 57
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.7548110958653614e-05
            Iterations: 215
            Function evaluations: 62566
            Gradient evaluations: 215
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5978355427899853
            Iterations: 49
            Function evaluations: 13139
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1886893305855933e-09
   

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0896213777467165e-09
            Iterations: 68
            Function evaluations: 21148
            Gradient evaluations: 68
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.6968829416809135e-05
            Iterations: 222
            Function evaluations: 69931
            Gradient evaluations: 222
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.342686556532796
            Iterations: 51
            Function evaluations: 14868
            Gradient evaluations: 47
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.095078213397084e-09
            Iterations: 56
            Function evaluations: 17640
            Gradient evaluations: 56
2009-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.917

2011-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.824305208054584e-05
            Iterations: 178
            Function evaluations: 58562
            Gradient evaluations: 178
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.000896281004322
            Iterations: 38
            Function evaluations: 11212
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.521019865595557e-10
            Iterations: 64
            Function evaluations: 21056
            Gradient evaluations: 64
2011-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.7831674572538414e-05
            Iterations: 229
            Function evaluations: 75571
            Gradient evaluations: 229
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9619839341383067
            Iterations: 88
            Function evaluations: 29419
            Gradient evaluations: 84
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2748449745563029e-09
            Iterations: 15
            Function evaluations: 5235
            Gradient evaluations: 15
2013-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7128904451588232e-05
            Iterations: 313
            Function evaluations: 109863
            Gradient evaluations: 313
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.072654167336018
            Iterations: 69
            Function evaluations: 22881
            Gradient evaluations: 65
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9008667446362355e-09
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6576086741878505e-09
            Iterations: 46
            Function evaluations: 17112
            Gradient evaluations: 46
2015-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.429249483371104e-05
            Iterations: 278
            Function evaluations: 103972
            Gradient evaluations: 278
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.909570732352028
            Iterations: 71
            Function evaluations: 25135
            Gradient evaluations: 67
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9700169084537776e-09
            Iterations: 44
            Function evaluations: 16456
            Gradient evaluations: 44
2015-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.18

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.162293138326003e-05
            Iterations: 335
            Function evaluations: 136680
            Gradient evaluations: 335
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.3109959880876403
            Iterations: 75
            Function evaluations: 29037
            Gradient evaluations: 71
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.697803903200421e-09
            Iterations: 9
            Function evaluations: 3672
            Gradient evaluations: 9
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.726437042699659e-05
            Iterations: 310
            Function evaluations: 126790
            Gradient evaluations: 310
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.18

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.058936735407987
            Iterations: 35
            Function evaluations: 13967
            Gradient evaluations: 33
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.05886249133464e-10
            Iterations: 80
            Function evaluations: 33761
            Gradient evaluations: 80
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00015730036926424058
            Iterations: 176
            Function evaluations: 74800
            Gradient evaluations: 176
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.08139264756494
            Iterations: 39
            Function evaluations: 14906
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0571410942232066e-09
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.704625662799398e-10
            Iterations: 56
            Function evaluations: 24976
            Gradient evaluations: 56
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.241555407363865e-05
            Iterations: 221
            Function evaluations: 98787
            Gradient evaluations: 221
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3736513283940344
            Iterations: 39
            Function evaluations: 15677
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0491657464190775e-09
            Iterations: 55
            Function evaluations: 24585
            Gradient evaluations: 55
2022-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.420

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3830694044519194e-05
            Iterations: 306
            Function evaluations: 142902
            Gradient evaluations: 306
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4.4353991787064535
            Iterations: 100
            Function evaluations: 44944
            Gradient evaluations: 96
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6403127585131448e-09
            Iterations: 20
            Function evaluations: 9340
            Gradient evaluations: 20
2024-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4297180957088156e-05
            Iterations: 262
            Function evaluations: 122092
            Gradient evaluations: 262
Positive directional derivative for linesearch    (Exit mode 8)
            Cur

In [62]:
#Exportieren nach CSV
portfolio_returns_df_shrinked.to_csv("returnssp500shrinked_2Monate_f.csv")

### Auswertung JSE mit Ledoit Wolf

In [63]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [64]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 2
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_JSE_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_JSE_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_JSE_gewichte_min_variance,
    "Maximum-Diversification": shrinked_JSE_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_JSE_gewichte_risk_parity,
    "HRP": shrinked_JSE_gewichte_hrp,
    "HERC": shrinked_JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked_JSE = portfolio_returns_df_shrinked_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.240047133151458e-05
            Iterations: 135
            Function evaluations: 9045
            Gradient evaluations: 135
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.741709713501023
            Iterations: 43
            Function evaluations: 2923
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.635081070068458e-09
            Iterations: 44
            Function evaluations: 2949
            Gradient evaluations: 44
2005-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.119348145512657e-05
            Iterations: 124
            Function evaluations: 8432
            Gradient evaluations: 124
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.314781

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.048045456084472
            Iterations: 44
            Function evaluations: 3436
            Gradient evaluations: 44
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.311800058952099e-09
            Iterations: 32
            Function evaluations: 2465
            Gradient evaluations: 32
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.0519928358813376e-05
            Iterations: 115
            Function evaluations: 9200
            Gradient evaluations: 115
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.24488714898468
            Iterations: 57
            Function evaluations: 4619
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.053327078609027e-09
            

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.270719421979131e-09
            Iterations: 65
            Function evaluations: 5590
            Gradient evaluations: 65
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.722853830987692e-05
            Iterations: 167
            Function evaluations: 14029
            Gradient evaluations: 167
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.6411501206122887
            Iterations: 52
            Function evaluations: 4417
            Gradient evaluations: 52
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.403606949005002e-09
            Iterations: 56
            Function evaluations: 4704
            Gradient evaluations: 56
2009-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3392120

2011-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2471539735383868e-05
            Iterations: 211
            Function evaluations: 19413
            Gradient evaluations: 211
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.61427278355533
            Iterations: 61
            Function evaluations: 5675
            Gradient evaluations: 61
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.4769257463180325e-09
            Iterations: 45
            Function evaluations: 4141
            Gradient evaluations: 45
2011-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.176701982135874e-06
            Iterations: 215
            Function evaluations: 19566
            Gradient evaluations: 215
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.332

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.7875201718638785
            Iterations: 79
            Function evaluations: 8244
            Gradient evaluations: 79
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.984646843731761e-09
            Iterations: 20
            Function evaluations: 2060
            Gradient evaluations: 20
2013-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.236705230782776e-06
            Iterations: 171
            Function evaluations: 17784
            Gradient evaluations: 171
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.936148628068598
            Iterations: 87
            Function evaluations: 9160
            Gradient evaluations: 87
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5188573104448916e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.878545054057707e-09
            Iterations: 170
            Function evaluations: 19890
            Gradient evaluations: 170
2015-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2652606574566897e-05
            Iterations: 176
            Function evaluations: 20593
            Gradient evaluations: 176
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.8716009033644045
            Iterations: 70
            Function evaluations: 7810
            Gradient evaluations: 66
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2662802774366502e-08
            Iterations: 92
            Function evaluations: 10764
            Gradient evaluations: 92
2015-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2621244000165396e-05
            Iterations: 179
            Function evaluations: 23450
            Gradient evaluations: 179
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.98667533013252
            Iterations: 102
            Function evaluations: 13388
            Gradient evaluations: 101
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.328044018515022e-09
            Iterations: 37
            Function evaluations: 4847
            Gradient evaluations: 37
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6109168058748373e-05
            Iterations: 182
            Function evaluations: 24570
            Gradient evaluations: 182
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.004599204643735
            Iterations: 71
            Function evaluations: 9734
            Gradient evaluations: 70
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.685769657042407e-10
            Iterations: 85
            Function evaluations: 11730
            Gradient evaluations: 85
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.958425790605818e-05
            Iterations: 135
            Function evaluations: 19170
            Gradient evaluations: 135
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.0454098129435065
            Iterations: 98
            Function evaluations: 13565
            Gradient evaluations: 94
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0178346886192908e-08
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.255661596469203e-09
            Iterations: 33
            Function evaluations: 4984
            Gradient evaluations: 33
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4585913732774674e-05
            Iterations: 165
            Function evaluations: 25080
            Gradient evaluations: 165
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.317671964533073
            Iterations: 94
            Function evaluations: 13798
            Gradient evaluations: 90
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0167320243649784e-09
            Iterations: 44
            Function evaluations: 6689
            Gradient evaluations: 44
2022-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.32654

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1218483406704243e-05
            Iterations: 182
            Function evaluations: 27664
            Gradient evaluations: 182
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.546489960492811
            Iterations: 117
            Function evaluations: 17817
            Gradient evaluations: 116
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.036273113821135e-09
            Iterations: 38
            Function evaluations: 5776
            Gradient evaluations: 38
2024-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.109424478719857e-05
            Iterations: 184
            Function evaluations: 27969
            Gradient evaluations: 184
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.

In [65]:
#Exportieren nach CSV
portfolio_returns_df_shrinked_JSE.to_csv("returnsJSEshrinked_2Monate_f.csv")

### Reduzierter S&P500-Bestand

In [66]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [67]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 2
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_gewichte_min_variance,
    "Maximum-Diversification": reduziert_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_gewichte_risk_parity,
    "HRP": reduziert_gewichte_hrp,
    "HERC": reduziert_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
   #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert = portfolio_returns_df_reduziert.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.105745668140832e-05
            Iterations: 279
            Function evaluations: 28738
            Gradient evaluations: 279
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5646187894305896
            Iterations: 48
            Function evaluations: 4992
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3432286542379153e-09
            Iterations: 21
            Function evaluations: 2163
            Gradient evaluations: 21
2005-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1773795573208203e-05
            Iterations: 278
            Function evaluations: 29190
            Gradient evaluations: 278
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.4198426399490716
            Iterations: 26
            Function evaluations: 3199
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.963852340402336e-10
            Iterations: 61
            Function evaluations: 7442
            Gradient evaluations: 61
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.96402179645009e-05
            Iterations: 177
            Function evaluations: 21594
            Gradient evaluations: 177
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.364275709503564
            Iterations: 28
            Function evaluations: 3440
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.5360053058636926e-10
          

Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.022232434998734e-10
            Iterations: 60
            Function evaluations: 8580
            Gradient evaluations: 60
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.2759582445145284e-05
            Iterations: 191
            Function evaluations: 27313
            Gradient evaluations: 191
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0462069905272684
            Iterations: 23
            Function evaluations: 3309
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.164847993443566e-10
            Iterations: 62
            Function evaluations: 8866
            Gradient evaluations: 62
2009-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.266302

2011-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.6212508961286646e-05
            Iterations: 148
            Function evaluations: 21312
            Gradient evaluations: 148
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6837778762517404
            Iterations: 23
            Function evaluations: 3326
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.465251881483289e-10
            Iterations: 68
            Function evaluations: 9792
            Gradient evaluations: 68
2011-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9351581306305057e-05
            Iterations: 230
            Function evaluations: 33120
            Gradient evaluations: 230
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.51095663880057
            Iterations: 38
            Function evaluations: 5738
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.986343755908541e-09
            Iterations: 20
            Function evaluations: 3000
            Gradient evaluations: 20
2013-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.967967663577138e-05
            Iterations: 256
            Function evaluations: 38400
            Gradient evaluations: 256
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.460878557187144
            Iterations: 41
            Function evaluations: 6190
            Gradient evaluations: 41
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8226126812923991e-09
           

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2334463598452176e-09
            Iterations: 47
            Function evaluations: 7943
            Gradient evaluations: 47
2015-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.755078480144621e-05
            Iterations: 262
            Function evaluations: 44279
            Gradient evaluations: 262
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5242144841808396
            Iterations: 39
            Function evaluations: 6630
            Gradient evaluations: 39
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4129191080936424e-09
            Iterations: 48
            Function evaluations: 8112
            Gradient evaluations: 48
2015-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.54475

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.516005193618761e-05
            Iterations: 341
            Function evaluations: 63767
            Gradient evaluations: 341
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8185518885586376
            Iterations: 45
            Function evaluations: 8459
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2821134077180467e-09
            Iterations: 20
            Function evaluations: 3740
            Gradient evaluations: 20
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1371516057450396e-05
            Iterations: 260
            Function evaluations: 48620
            Gradient evaluations: 260
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8854257950472955
            Iterations: 25
            Function evaluations: 4988
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.289622825630533e-10
            Iterations: 104
            Function evaluations: 20697
            Gradient evaluations: 104
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001738678734999268
            Iterations: 162
            Function evaluations: 32239
            Gradient evaluations: 162
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.961729977689875
            Iterations: 20
            Function evaluations: 3992
            Gradient evaluations: 20
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.366140755582507e-10
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6348088524982074e-09
            Iterations: 47
            Function evaluations: 9635
            Gradient evaluations: 47
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.231610610649447e-05
            Iterations: 248
            Function evaluations: 50841
            Gradient evaluations: 248
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.202603291143996
            Iterations: 33
            Function evaluations: 6797
            Gradient evaluations: 33
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.936401038241577e-10
            Iterations: 58
            Function evaluations: 11890
            Gradient evaluations: 58
2022-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.501780

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8104246525629658e-05
            Iterations: 305
            Function evaluations: 64356
            Gradient evaluations: 305
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7842843929033183
            Iterations: 68
            Function evaluations: 13566
            Gradient evaluations: 64
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5028171951972646e-09
            Iterations: 33
            Function evaluations: 6963
            Gradient evaluations: 33
2024-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.912688213821159e-05
            Iterations: 298
            Function evaluations: 64667
            Gradient evaluations: 298
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.

In [68]:
#Exportieren nach CSV
portfolio_returns_df_reduziert.to_csv("returnsreduziert_2Monate_f.csv")

### Reduziert und Ledoit-Shrinkage

In [69]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [70]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 2
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_shrinked_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_shrinked_gewichte_min_variance,
    "Maximum-Diversification": reduziert_shrinked_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_shrinked_gewichte_risk_parity,
    "HRP": reduziert_shrinked_gewichte_hrp,
    "HERC": reduziert_shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert_shrinked = portfolio_returns_df_reduziert_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.068558709922772e-05
            Iterations: 262
            Function evaluations: 26986
            Gradient evaluations: 262
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6511078138392
            Iterations: 48
            Function evaluations: 4991
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9801819573744985e-09
            Iterations: 20
            Function evaluations: 2060
            Gradient evaluations: 20
2005-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.182482490877974e-05
            Iterations: 240
            Function evaluations: 25200
            Gradient evaluations: 240
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.57275

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.4501496718585263
            Iterations: 29
            Function evaluations: 3568
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.5274305668087737e-10
            Iterations: 61
            Function evaluations: 7442
            Gradient evaluations: 61
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.902931173855929e-05
            Iterations: 161
            Function evaluations: 19642
            Gradient evaluations: 161
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3996157677387666
            Iterations: 27
            Function evaluations: 3322
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.57058040759739e-10
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.565722637539197e-10
            Iterations: 60
            Function evaluations: 8580
            Gradient evaluations: 60
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.242193406763155e-05
            Iterations: 198
            Function evaluations: 28314
            Gradient evaluations: 198
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0685705882687615
            Iterations: 25
            Function evaluations: 3596
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.891643572263232e-10
            Iterations: 62
            Function evaluations: 8867
            Gradient evaluations: 62
2009-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.2485740

2011-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.606116173794853e-05
            Iterations: 132
            Function evaluations: 19008
            Gradient evaluations: 132
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7173518295781094
            Iterations: 23
            Function evaluations: 3329
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.18270574388655e-10
            Iterations: 69
            Function evaluations: 9936
            Gradient evaluations: 69
2011-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9387749952384117e-05
            Iterations: 191
            Function evaluations: 27504
            Gradient evaluations: 191
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.129

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5504625286005402
            Iterations: 44
            Function evaluations: 6642
            Gradient evaluations: 44
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8606543118409818e-09
            Iterations: 20
            Function evaluations: 3000
            Gradient evaluations: 20
2013-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9065299371316638e-05
            Iterations: 271
            Function evaluations: 40651
            Gradient evaluations: 271
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5041930369138656
            Iterations: 42
            Function evaluations: 6340
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7009845074522453e-09
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1758259491185698e-09
            Iterations: 47
            Function evaluations: 7943
            Gradient evaluations: 47
2015-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.651827435344933e-05
            Iterations: 273
            Function evaluations: 46137
            Gradient evaluations: 273
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5536665817899604
            Iterations: 35
            Function evaluations: 5950
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3325306700801538e-09
            Iterations: 48
            Function evaluations: 8112
            Gradient evaluations: 48
2015-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.45418

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3827371940727167e-05
            Iterations: 293
            Function evaluations: 54791
            Gradient evaluations: 293
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.880935273736616
            Iterations: 54
            Function evaluations: 10152
            Gradient evaluations: 54
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.068492382711447e-09
            Iterations: 20
            Function evaluations: 3740
            Gradient evaluations: 20
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9993509539989655e-05
            Iterations: 259
            Function evaluations: 48433
            Gradient evaluations: 259
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.7

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9399200897438789
            Iterations: 23
            Function evaluations: 4596
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.56759010112444e-10
            Iterations: 105
            Function evaluations: 20896
            Gradient evaluations: 105
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001706362963578513
            Iterations: 139
            Function evaluations: 27661
            Gradient evaluations: 139
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.002074512397377
            Iterations: 23
            Function evaluations: 4592
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.716708726828573e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5635656995424195e-09
            Iterations: 47
            Function evaluations: 9636
            Gradient evaluations: 47
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.120665236936547e-05
            Iterations: 250
            Function evaluations: 51250
            Gradient evaluations: 250
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.228138697365681
            Iterations: 32
            Function evaluations: 6592
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.562408509300391e-10
            Iterations: 58
            Function evaluations: 11890
            Gradient evaluations: 58
2022-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.382679

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7142097511704284e-05
            Iterations: 251
            Function evaluations: 52962
            Gradient evaluations: 251
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8899534090777754
            Iterations: 68
            Function evaluations: 14419
            Gradient evaluations: 68
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.978264312522957e-09
            Iterations: 20
            Function evaluations: 4221
            Gradient evaluations: 20
2024-02-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8144847695536962e-05
            Iterations: 263
            Function evaluations: 57071
            Gradient evaluations: 263
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.

In [71]:
#Exportieren nach CSV
portfolio_returns_df_reduziert_shrinked.to_csv("returnsreduziertshrinked_2Monate_f.csv")